In [1]:
import os
path = "/Users/patricia/Documents/code/python-code/behavior-detection/src"
os.chdir(path)  # Muda o diretório para o nível anterior (a raiz do projeto)
print(os.getcwd())  # Verifique se agora está na raiz

/Users/patricia/Documents/code/python-code/behavior-detection/src


# Load data

In [2]:
from behavior.data.behavior_data_loader import BehaviorDataLoader

data_path = '../data/new_logs_labels.csv'

data = BehaviorDataLoader.load_data(data_path, delimiter=';')
print(data.shape)
data.head(5)

(5525, 372)


,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [3]:
from core.preprocessors.data_cleaner import DataCleaner

print("Valores da coluna 'comportamento' antes da remoção:", data['comportamento'].value_counts())

# Remove instances where 'comportamento' is '?'
data = DataCleaner.remove_instances_with_value(data, 'comportamento', '?')

print("\nValores da coluna 'comportamento' depois da remoção:", data['comportamento'].value_counts())

Valores da coluna 'comportamento' antes da remoção: comportamento
ON TASK                 3159
ON SYSTEM                907
OFF TASK                 629
ON TASK CONVERSATION     414
ON TASK OUT              380
?                         36
Name: count, dtype: int64

Valores da coluna 'comportamento' depois da remoção: comportamento
ON TASK                 3159
ON SYSTEM                907
OFF TASK                 629
ON TASK CONVERSATION     414
ON TASK OUT              380
Name: count, dtype: int64


In [4]:
data.head(5)

,id_log,aluno,grupo,num_dia,num_log,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,...,comportamento_off_task,comportamento_on_system,comportamento_indefinido,ultimo_comportamento,ultimo_comportamento_on_task,ultimo_comportamento_on_task_conversation,ultimo_comportamento_on_task_out,ultimo_comportamento_off_task,ultimo_comportamento_on_system,ultimo_comportamento_indefinido
0,8224,1,2,1,1,step_verification,0,0,1,0,...,0,0,0,?,0,0,0,0,0,1
1,527786,1,2,1,2,user_idle,1,0,0,2,...,0,0,0,ON TASK,1,0,0,0,0,0
2,527787,1,2,1,3,user_idle,0,0,0,4,...,0,0,0,ON TASK,1,0,0,0,0,0
3,527788,1,2,1,4,user_idle,0,0,0,6,...,0,0,0,ON TASK,1,0,0,0,0,0
4,527789,1,2,1,5,user_idle,0,0,0,8,...,0,0,0,ON TASK,1,0,0,0,0,0


In [5]:
from sklearn.model_selection import train_test_split

# Select a subset of the data only for testing purposes

print("Tamanho do dataframe antes:", data.shape)
data, _ = train_test_split(data, test_size=0.2, stratify=data['comportamento'], random_state=42)
data.reset_index(drop=True, inplace=True)
print("Tamanho do dataframe após:", data.shape)

Tamanho do dataframe antes: (5489, 372)
Tamanho do dataframe após: (4391, 372)


# Pre-processing

## Remove unnecessary columns

In [6]:
# Removing columns related to IDs, emotions, personality and behaviors, because 
# we want to classify behaviors only by the students' interactions with the system
columns_to_remove_ids = ['id_log', 'grupo', 'num_dia', 'num_log']
columns_to_remove_emotions = [
    'estado_afetivo', 'estado_engajamento_concentrado', 
    'estado_confusao', 'estado_frustracao', 'estado_tedio', 'estado_indefinido', 
    'ultimo_estado_afetivo', 'ultimo_engajamento_concentrado', 'ultimo_confusao', 
    'ultimo_frustracao', 'ultimo_tedio', 'ultimo_estado_indefinido'
]
columns_to_remove_personality = [
    'traco_amabilidade_fator', 'traco_extrovercao_fator', 'traco_conscienciosidade_fator', 
    'traco_abertura_fator', 'traco_neuroticismo_fator', 'traco_amabilidade_cat', 
    'traco_extrovercao_cat', 'traco_conscienciosidade_cat', 'traco_abertura_cat', 
    'traco_neuroticismo_cat']

columns_to_remove_behaviors = [
    'comportamento_on_task', 'comportamento_on_task_conversation', 'comportamento_on_task_out',
    'comportamento_off_task', 'comportamento_on_system', 'comportamento_indefinido',
    'ultimo_comportamento', 'ultimo_comportamento_on_task', 'ultimo_comportamento_on_task_conversation',
    'ultimo_comportamento_on_task_out', 'ultimo_comportamento_off_task', 'ultimo_comportamento_on_system',
    'ultimo_comportamento_indefinido'
]

columns_to_remove = columns_to_remove_ids + \
        columns_to_remove_emotions + \
        columns_to_remove_personality + \
        columns_to_remove_behaviors

cleaned_data = DataCleaner.remove_columns(data, columns_to_remove)


In [7]:
cleaned_data.head(5)

,aluno,log_type,ultimo_passo_correto,verificado_com_mouse,verificado_com_teclado,idle_time_acumulado,num_click_acumulado,num_click_passo,num_click_eq,type_step_verification,...,misc_OI_Dv_Plus_Sb_total,misc_EqSec_Distrib_MtTerm_total,misc_OI_Mt_Minus_Mt_Plus_total,misc_OI_Mt_Minus_Mt_Minus_total,misc_OI_Dv_Plus_Ad_total,misc_EqPrim_Mt_Inc_total,misc_EqPrim_Dv_Inc_total,misc_OI_Dv_Minus_Dv_Minus_total,misc_EqSec_OpFrac_MMC_MtNumerador_total,comportamento
0,27,user_idle,0,0,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ON TASK
1,16,user_idle,0,0,0,10,0,0,1,0,...,0,0,0,0,0,0,1,0,0,ON TASK OUT
2,11,click,0,0,0,0,1,3,5,0,...,0,0,0,0,0,0,1,0,0,ON TASK
3,28,user_idle,0,0,0,4,0,1,1,0,...,0,0,0,0,0,0,0,0,0,ON TASK OUT
4,13,user_idle,0,0,0,10,0,2,2,0,...,0,0,0,0,0,0,0,0,0,OFF TASK


In [8]:
# Preenche valores ausentes no DataFrame X com a string 'missing'.

numeric_columns = cleaned_data.select_dtypes(include=['float64', 'int64']).columns
categorical_columns = cleaned_data.select_dtypes(exclude=['float64', 'int64']).columns

cleaned_data[numeric_columns] = cleaned_data[numeric_columns].fillna(cleaned_data[numeric_columns].median())
cleaned_data[categorical_columns] = cleaned_data[categorical_columns].fillna('missing')


## Split data by student level into training and test datasets

In [9]:
from core.preprocessors.data_splitter import DataSplitter

train_data, test_data = DataSplitter.split_by_student_level(cleaned_data, test_size=0.2, column_name='aluno')

In [10]:
# 2. Após o split por nível do estudante
print("\n=== Após split por nível do estudante ===")
print(f"Shape de train_data: {train_data.shape}")
print("Colunas em train_data:", train_data.columns.tolist())


=== Após split por nível do estudante ===
Shape de train_data: (3529, 333)
Colunas em train_data: ['aluno', 'log_type', 'ultimo_passo_correto', 'verificado_com_mouse', 'verificado_com_teclado', 'idle_time_acumulado', 'num_click_acumulado', 'num_click_passo', 'num_click_eq', 'type_step_verification', 'type_click', 'type_typing', 'type_mouse_stop', 'type_mouse_fast', 'type_hint_request', 'type_user_idle', 'type_window_lost_focus', 'type_window_gained_focus', 'type_window_resize', 'type_left_window', 'type_entered_window', 'type_start_out_of_sinc_server_time', 'type_user_log_out', 'click_resolucao', 'click_planos', 'click_atalhos', 'click_bugs', 'click_gamificacao', 'click_dicas', 'click_erros', 'click_notificacoes', 'click_sistema', 'click_emocoes', 'click_indefinidos', 'pontuacao', 'cat_muito_facil', 'cat_facil_1', 'cat_facil_2', 'cat_facil_3', 'cat_facil_4', 'cat_medio_1', 'cat_medio_2', 'cat_medio_3', 'cat_dificil_1', 'cat_dificil_2', 'cat_dificil_3', 'cat_dificil_4', 'num_passos_equ

In [11]:
# removing the 'aluno' column from the data after splitting into train and test sets

# Remover 'aluno' do conjunto de treinamento
train_data = DataCleaner.remove_columns(train_data, ['aluno'])

# Remover 'aluno' do conjunto de teste
test_data = DataCleaner.remove_columns(test_data, ['aluno'])

In [12]:
# 1. Após remover coluna 'aluno'
print("\n1. Após remover 'aluno':")
print(f"Shape de train_data: {train_data.shape}")


1. Após remover 'aluno':
Shape de train_data: (3529, 332)


## Split data into Features (X) and Target (y)

In [13]:
from core.preprocessors.data_splitter import DataSplitter

# Conjunto de treinamento
X_train, y_train = DataSplitter.split_into_x_y(train_data, 'comportamento')

# Conjunto de teste
X_test, y_test = DataSplitter.split_into_x_y(test_data, 'comportamento')

In [14]:
import pandas as pd
import numpy as np
# 2. Após split X/y
print("\n2. Após split X/y:")
if isinstance(X_train, pd.DataFrame):
    print(f"Shape de X_train: {X_train.shape}")
    print("Primeiras colunas de X_train:", list(X_train.columns)[:5])
else:
    print("X_train não é um DataFrame!")
    print(f"Tipo de X_train: {type(X_train)}")


2. Após split X/y:
Shape de X_train: (3529, 331)
Primeiras colunas de X_train: ['log_type', 'ultimo_passo_correto', 'verificado_com_mouse', 'verificado_com_teclado', 'idle_time_acumulado']


In [15]:
print("Primeiras 5 instâncias de y_train:")
print(y_train[:5])

print("\nPrimeiras 5 instâncias de y_test:")
print(y_test[:5])

Primeiras 5 instâncias de y_train:
0        ON TASK
1    ON TASK OUT
2        ON TASK
3    ON TASK OUT
4       OFF TASK
Name: comportamento, dtype: object

Primeiras 5 instâncias de y_test:
11    ON TASK
12    ON TASK
22    ON TASK
24    ON TASK
31    ON TASK
Name: comportamento, dtype: object


## Encoding variables

### Encoding true labels (y)

In [16]:
import importlib
from core.preprocessors import column_selector, data_encoder
from behavior.data import behavior_data_encoder

# Recarregar o módulo para garantir que as alterações sejam aplicadas
importlib.reload(column_selector)
importlib.reload(data_encoder)
importlib.reload(behavior_data_encoder)

<module 'behavior.data.behavior_data_encoder' from '/Users/patricia/Documents/code/python-code/behavior-detection/src/behavior/data/behavior_data_encoder.py'>

In [17]:
# Encoding y_train and y_test
from behavior.data.behavior_data_encoder import BehaviorDataEncoder

# Codificar y_train
y_train = BehaviorDataEncoder.encode_y(y_train)

# Codificar y_test
y_test = BehaviorDataEncoder.encode_y(y_test)



### Encoding features (X)

In [18]:
# Importações necessárias
import pandas as pd
import numpy as np
from behavior.data.behavior_data_encoder import BehaviorDataEncoder

# Encoding do target (y)
y_train = BehaviorDataEncoder.encode_y(y_train)
y_test = BehaviorDataEncoder.encode_y(y_test)

# Encoding das features (X)
print("=== Iniciando encoding das features ===")
X_encoder = BehaviorDataEncoder(num_classes=5)
print("\nRealizando fit do encoder...")
X_encoder.fit(X_train)

print("\nRealizando transform...")
X_train = X_encoder.transform(X_train)

print("\nTransformando dados de teste...")
X_test = X_encoder.transform(X_test)

# Verificação final
print("\n=== Verificação após encoding ===")
print(f"Shape de X_train: {X_train.shape}")
print(f"Shape de X_test: {X_test.shape}")
print(f"Shape de y_train: {y_train.shape}")
print(f"Shape de y_test: {y_test.shape}")

=== Iniciando encoding das features ===

Realizando fit do encoder...
Entrada fit - Shape: (3529, 331)
Fit realizado com sucesso
Colunas numéricas: 330
Colunas nominais: 179

Realizando transform...
Entrada transform - Shape: (3529, 331)
Saída transform - Shape: (3529, 466)

Transformando dados de teste...
Entrada transform - Shape: (862, 331)
Saída transform - Shape: (862, 466)

=== Verificação após encoding ===
Shape de X_train: (3529, 466)
Shape de X_test: (862, 466)
Shape de y_train: (3529,)
Shape de y_test: (862,)


/opt/anaconda3/envs/behavior_detection/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [23, 37, 38, 41, 51, 52, 55, 56, 57, 60, 62, 67, 68, 74, 78, 82, 90, 97, 113, 131, 132, 141, 153, 161, 162, 167, 170, 174, 178] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [19]:
print(X_test.shape)
print(X_test.head(10))

(862, 466)
    num_standard__ultimo_passo_correto  num_standard__verificado_com_mouse  \
11                           -0.233189                           -0.060806   
12                           -0.233189                           -0.060806   
22                           -0.233189                           -0.060806   
24                           -0.233189                           -0.060806   
31                           -0.233189                           -0.060806   
34                           -0.233189                           -0.060806   
43                           -0.233189                           -0.060806   
50                           -0.233189                           -0.060806   
53                           -0.233189                           -0.060806   
59                           -0.233189                           -0.060806   

    num_standard__verificado_com_teclado  num_standard__idle_time_acumulado  \
11                             -0.268315           

In [20]:
# Antes do SMOTE, adicione estas verificações
print("Verificando X_train antes do SMOTE:")
print("1. Shape de X_train:", X_train.shape)
print("2. Tipo de X_train:", type(X_train))
print("3. Shape de y_train:", y_train.shape)
print("4. Tipo de y_train:", type(y_train))

if isinstance(X_train, pd.DataFrame):
    print("5. Colunas em X_train:")
    print(X_train.columns.tolist())
    print("\n6. Primeiras linhas de X_train:")
    print(X_train.head())
    print("\n7. Tipos de dados das colunas:")
    print(X_train.dtypes)

Verificando X_train antes do SMOTE:
1. Shape de X_train: (3529, 466)
2. Tipo de X_train: <class 'pandas.core.frame.DataFrame'>
3. Shape de y_train: (3529,)
4. Tipo de y_train: <class 'numpy.ndarray'>
5. Colunas em X_train:
['num_standard__ultimo_passo_correto', 'num_standard__verificado_com_mouse', 'num_standard__verificado_com_teclado', 'num_standard__idle_time_acumulado', 'num_standard__num_click_acumulado', 'num_standard__num_click_passo', 'num_standard__num_click_eq', 'num_standard__type_step_verification', 'num_standard__type_click', 'num_standard__type_typing', 'num_standard__type_mouse_stop', 'num_standard__type_mouse_fast', 'num_standard__type_hint_request', 'num_standard__type_user_idle', 'num_standard__type_window_lost_focus', 'num_standard__type_window_gained_focus', 'num_standard__type_window_resize', 'num_standard__type_left_window', 'num_standard__type_entered_window', 'num_standard__type_start_out_of_sinc_server_time', 'num_standard__type_user_log_out', 'num_standard__cl

# Balanceamento dos dados

In [21]:
from core.preprocessors.data_balancer import DataBalancer

data_balancer = DataBalancer()
X_train, y_train = data_balancer.apply_smote(X_train, y_train)

In [22]:
from collections import Counter

print(f"Resampled dataset shape: {Counter(y_train)}")

Resampled dataset shape: Counter({2: 1935, 4: 1935, 0: 1935, 1: 1935, 3: 1935})


# Treinamento dos Modelos

## Definindo parametros

In [23]:
# Importações e configuração de diretório permanecem iguais até a seção de treinamento

# Na seção "Definindo parametros", substituir:
from core.models.multiclass.behavior_model_params import BehaviorModelParams

# Criar instância dos parâmetros específicos para comportamentos
model_params = BehaviorModelParams()

# # Definir quais modelos e seletores utilizar
# selected_models = [ 
#     # 'Logistic Regression',
#     'Decision Tree',
#     # 'Random Forest',
#     # 'Gradient Boosting',
#     # 'SVM',
#     # 'KNN',
#     # 'XGBoost',
#     'Naive Bayes' 
#     # 'MLP'  
# ]

# # Definir quais seletores de features utilizar
# selected_selectors = [
#     # 'rfe',      # Recursive Feature Elimination
#     'pca',      # Principal Component Analysis
#     # 'rf',       # Random Forest Feature Selector
#     # 'mi',       # Mutual Information Feature Selector
#     'none'      # Sem seleção de features
# ]


# # Usar todos os modelos disponíveis
selected_models = model_params.get_available_models()  # ou lista específica

# # Usar todos os seletores disponíveis
selected_selectors = None  # None to use all selectors

# Configurar validação cruzada estratificada
from sklearn.model_selection import StratifiedKFold
cv = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# Parâmetros de otimização
n_iter = 50  # Reduzido para teste inicial
n_jobs = 6  # MacBook Air M2 tem 8 núclos CPUs e 10 GPUs. Como uso sciktlearn, só posso usar CPUs. Teria que usar Pytorch ou TensorFlow para usar GPUs
scoring_metric = 'balanced_accuracy'



## Usando Otimização Bayesiana (Optuna)

In [24]:
from core.management.stage_training_manager import StageTrainingManager
from core.training.optuna_bayesian_optimization_training import OptunaBayesianOptimizationTraining

# Definir as etapas incorporando todos os seletores de features
stages = [
        ('etapa_1_logistic_none', ['Logistic Regression'], ['none']),
        ('etapa_2_logistic_pca', ['Logistic Regression'], ['pca']),
        ('etapa_3_logistic_rfe', ['Logistic Regression'], ['rfe']),
        ('etapa_4_logistic_rf', ['Logistic Regression'], ['rf']),
        ('etapa_5_logistic_mi', ['Logistic Regression'], ['mi']),
        ('etapa_6_tree_none', ['Decision Tree'], ['none']),
        ('etapa_7_tree_pca', ['Decision Tree'], ['pca']),
        ('etapa_8_tree_rfe', ['Decision Tree'], ['rfe']),
        ('etapa_9_tree_rf', ['Decision Tree'], ['rf']),
        ('etapa_10_tree_mi', ['Decision Tree'], ['mi']),
        ('etapa_11_rf_none', ['Random Forest'], ['none']),
        ('etapa_12_rf_pca', ['Random Forest'], ['pca']),
        ('etapa_13_rf_rfe', ['Random Forest'], ['rfe']),
        ('etapa_14_rf_rf', ['Random Forest'], ['rf']),
        ('etapa_15_rf_mi', ['Random Forest'], ['mi']),
        ('etapa_16_gb_none', ['Gradient Boosting'], ['none']),
        ('etapa_17_gb_pca', ['Gradient Boosting'], ['pca']),
        ('etapa_18_gb_rfe', ['Gradient Boosting'], ['rfe']),
        ('etapa_19_gb_rf', ['Gradient Boosting'], ['rf']),
        ('etapa_20_gb_mi', ['Gradient Boosting'], ['mi']),
        ('etapa_21_svm_none', ['SVM'], ['none']),
        ('etapa_22_svm_pca', ['SVM'], ['pca']),
        ('etapa_23_svm_rfe', ['SVM'], ['rfe']),
        ('etapa_24_svm_rf', ['SVM'], ['rf']),
        ('etapa_25_svm_mi', ['SVM'], ['mi']),
        ('etapa_26_knn_none', ['KNN'], ['none']),
        ('etapa_27_knn_pca', ['KNN'], ['pca']),
        ('etapa_28_knn_rfe', ['KNN'], ['rfe']),
        ('etapa_29_knn_rf', ['KNN'], ['rf']),
        ('etapa_30_knn_mi', ['KNN'], ['mi']),
        ('etapa_31_xgb_none', ['XGBoost'], ['none']),
        ('etapa_32_xgb_pca', ['XGBoost'], ['pca']),
        ('etapa_33_xgb_rfe', ['XGBoost'], ['rfe']),
        ('etapa_34_xgb_rf', ['XGBoost'], ['rf']),
        ('etapa_35_xgb_mi', ['XGBoost'], ['mi']),
        ('etapa_36_nb_none', ['Naive Bayes'], ['none']),
        ('etapa_37_nb_pca', ['Naive Bayes'], ['pca']),
        ('etapa_38_nb_rfe', ['Naive Bayes'], ['rfe']),
        ('etapa_39_nb_rf', ['Naive Bayes'], ['rf']),
        ('etapa_40_nb_mi', ['Naive Bayes'], ['mi']),
        ('etapa_41_mlp_none', ['MLP'], ['none']),
        ('etapa_42_mlp_pca', ['MLP'], ['pca']),
        ('etapa_43_mlp_rfe', ['MLP'], ['rfe']),
        ('etapa_44_mlp_rf', ['MLP'], ['rf']),
        ('etapa_45_mlp_mi', ['MLP'], ['mi'])
    ]

# Inicializar o gerenciador de treinamento
training_manager = StageTrainingManager(
    X_train=X_train,
    X_test=X_test,
    y_train=y_train,
    y_test=y_test,
    model_params=model_params,
    n_iter=50,
    cv=cv,
    scoring=scoring_metric,
    n_jobs=n_jobs
)

/opt/anaconda3/envs/behavior_detection/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
try:
    training_manager.execute_all_stages(training_manager, stages)
except Exception as e:
    print(f"\nExecução interrompida: {str(e)}")
    print("Você pode executar novamente o mesmo código para retomar do último stage não completado.")

2025-01-10 13:36:42,414 | optuna_training | INFO | Training and evaluating Logistic Regression with Optuna Optimization and none
INFO:optuna_training:Training and evaluating Logistic Regression with Optuna Optimization and none



Verificando progresso anterior...
Nenhum stage completado anteriormente. Iniciando do começo.

Iniciando Stage 1: etapa_1_logistic_none
Executando etapa: etapa_1_logistic_none
Modelos: ['Logistic Regression']
Seletores: ['none']
Iniciando treinamento da etapa etapa_1_logistic_none...
Inside OptunaBayesianOptimizationTraining.optimize_model


[W 2025-01-10 13:37:08,096] Trial 2 failed with parameters: {'classifier__penalty': 'l2', 'classifier__C': 10.0, 'classifier__solver': 'newton-cg', 'classifier__max_iter': 5000, 'classifier__class_weight': 'balanced'} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/anaconda3/envs/behavior_detection/lib/python3.10/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
  File "/Users/patricia/Documents/code/python-code/behavior-detection/src/core/training/optuna_bayesian_optimization_training.py", line 36, in objective
    return cross_val_score(
  File "/opt/anaconda3/envs/behavior_detection/lib/python3.10/site-packages/sklearn/utils/_param_validation.py", line 213, in wrapper
    return func(*args, **kwargs)
  File "/opt/anaconda3/envs/behavior_detection/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 712, in cross_val_score
    cv_results = cross_validate(
  Fil

KeyboardInterrupt: 

# Avaliação e logging

In [ ]:
from core.reporting import metrics_reporter

# Após todas as etapas estarem concluídas:
final_results = training_manager.combine_results()
training_results, class_metrics, avg_metrics = final_results
metrics_reporter.generate_reports(class_metrics, avg_metrics, filename_prefix="_Final_Combined_")